# Aprendizaje NO Supervisado

# Clustering

In [ ]:
import seaborn as sns
from sklearn import preprocessing
from sklearn import cluster
from sklearn.cluster import KMeans
from sklearn import preprocessing
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
import os
warnings.filterwarnings("ignore")

pd.set_option('display.float_format', lambda x: '%.2f' % x)

df = pd.read_csv("../data/2008_small.zip",nrows=100000)
df = df.sample(frac = 1)

### Tipos distintos de clustering
https://towardsdatascience.com/the-5-clustering-algorithms-data-scientists-need-to-know-a36d136ef68 

# K-Means
https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

Kmeans maximiza la varianza intergrupal, y minimiza la varianza intragrupal

In [ ]:
newdf = df[["AirTime","DepDelay"]].dropna()
# newdf = df[["AirTime","Distance","TaxiOut","TaxiIn","ArrDelay","DepDelay"]].dropna()

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=0).fit(newdf)

kmeans

In [ ]:
kmeans.labels_

In [ ]:
plt.scatter(newdf["AirTime"],newdf["DepDelay"],c = kmeans.labels_)
plt.xlabel("AirTime")
plt.ylabel("DepDelay")
plt.show()

>Podemos obtener información sobre los centros de cada uno de los clusters

In [ ]:
# ! pip install --upgrade scipy --user
import scipy
scipy.__version__

In [ ]:
from scipy.stats  import itemfreq

itemfreq(kmeans.labels_)

In [ ]:
kmeans.cluster_centers_

>Y hasta predicciones usando **datos nuevos**

In [ ]:
alldf = pd.read_csv("../data/2008_small.zip",nrows=1000000)[["AirTime","DepDelay"]].dropna()

In [ ]:
kmeans.predict(alldf) # "clasificar" nuevos puntos usando lo aprendido con los primeros 100.000 casos

In [ ]:
itemfreq(kmeans.predict(alldf))

>Ahora vamos a hacer el proceso con los datos reescalados, hasta añadiendo más columnas de las que podemos visualizar

In [ ]:
df = df[["AirTime","DepDelay","ArrDelay","Distance"]].dropna()
X_scaled = preprocessing.scale(df)

kmeans = KMeans(n_clusters=7, random_state=0).fit(X_scaled)

X_scaled # (x - media) / sd

In [ ]:
plt.figure(figsize=(10,15))
plt.scatter(df["AirTime"],df["DepDelay"],c = kmeans.labels_)
plt.show()

> `Y = (X - media) / desviacion`\
> `X = Y * desviacion + media`

In [ ]:
pd.DataFrame(kmeans.cluster_centers_, columns = df.columns)

In [ ]:
pd.DataFrame( kmeans.cluster_centers_ * np.array(df.std()) + np.array(df.mean()), columns = df.columns)

# Hierarchical Cluster
http://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html
- Affinity
- Linkage

In [ ]:
df = pd.read_csv("../data/2008_small.zip",nrows=5000)
newdf = df[["AirTime","Distance","TaxiOut","ArrDelay","DepDelay"]].dropna()

In [ ]:
clstr = cluster.AgglomerativeClustering(n_clusters=5)
clstr.fit(newdf)
clstr.labels_

In [ ]:
clstr = cluster.AgglomerativeClustering(n_clusters=5)
clstr.fit_predict(newdf) # "Entrena" y muestra etiquetas directamente

In [ ]:
plt.scatter(newdf["AirTime"],newdf["Distance"],c = clstr.fit_predict(newdf))
plt.show()

In [ ]:
newdf["Agrupacion"] = clstr.labels_
newdf.head()

In [ ]:
newdf.groupby("Agrupacion").describe()

# Ejercicios Clustering

1. Ajusta un modelo de K-means y un modelo de clúster jerárquico para los datos "iris". ¿Qué cantidad de clusters te parece razonable en este caso?



2. Visualiza los datos de uno de los dos métodos de clúster, coloreando según su grupo

3. ¿Cuál de los dos algoritmos da resultados más equilibrados (tamaños de los grupos) con estos datos?

4. ¿Funciona mejor el modelo con los datos reescalados?

5. Compara los grupos que has generado con las especies, tienen sentido? Como lo representaríamos?

6. Explora algún algoritmo de clustering distinto a los que ya hemos visto, y aplicalo sobre iris o los datos de los vuelos

7. Usando un bucle, estudia cuánto tarda en ajustarse un Kmeans y una agrupación jerárquica para distintas cantidades de datos (100,1000,10000,100000, por ejemplo)

### Clustering II

1. Intenta agrupar los coches usando el algoritmo DBSCAN

2. Representa gráficamente los resultados, qué cantidad de grupos te parecen razonables?

3. Juega con el parámetro eps a ver qué diferencias se observan

4. Compara con los demás algoritmos de clústering que hemos visto!

5. Hay alguna correspondencia entre los grupos generados y la variable "origin"?

© Netmind S.L.

Todos los derechos reservados. Este documento (v1.00) ha sido diseñado para el uso exclusivo del cliente que atiende a esta formación.

Ninguna parte de este documento puede ser reproducida, distribuida o transmitida en cualquier forma o por cualquier medio sin el permiso previo por escrito de Netmind.